In [4]:
from datasets import Features, Value, Sequence, Image, DatasetInfo, Dataset
from tqdm import tqdm
import os
from datasets import load_dataset, concatenate_datasets
import json
import re
from copy import deepcopy

import asyncio
import math
import numpy as np
import random
import pandas as pd
from ollama import AsyncClient

from random import sample
from collections import Counter
from openai import OpenAI

from workdir.utils import save_json, load_json

from random import shuffle, sample

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setting Features and rows for each dataset

In [ ]:
# AgriExam_features = Features({
    
#  'id': Value(dtype='string', id=None),
#  'question': Value(dtype='string', id=None),
#  'options': Sequence(feature=Value("string")),
#  'answer': Value(dtype='string', id=None),
 
#  'category': Value(dtype='string', id=None), # Plant Science, Pests, Taxonomy, Scientific_name
#  'question_type': Value(dtype='string', id=None), # multiple choice, open ended 
 
#  'metadata':{'source': Value(dtype='string'),
#                               'license': Value(dtype='string'),
#                               'url': Value(dtype='string'),
#                               'language': Value(dtype='string'),
#                               'verbose_answer': Value(dtype='string') 
#                               }
                    
#  })

In [ ]:
# Agri500P_features = Features({
    
#  'id': Value(dtype='string', id=None),
#  'question': Value(dtype='string', id=None),
#  'answer': Value(dtype='string', id=None),
 
#  'context': Value(dtype='string', id=None),
 
#  'category': Value(dtype='string', id=None), # Plant Science, Pests, Taxonomy
#  'question_type': Value(dtype='string', id=None), # multiple choice, open ended 
 
#  'metadata': {'source': Value(dtype='string'),
#                                'license': Value(dtype='string'),
#                                'language': Value(dtype='string'),
#                                'book_title': Value(dtype='string'),
#                                'chapter_title': Value(dtype='string')
#                                }
#  })

In [35]:
Agri500P_row = {
    "id": "_Agri500P_{number}",
    "question": "{question}",
    "answer": "{answer}",
    "context": "{context}",
    "category": "{category}",  # Example: Plant Science, Pests, Taxonomy
    "question_type": "{question_type}",  # Example: multiple choice, open ended
    "metadata":
        {
            "source": "{source}",
            "license": "{license}",
            "language": "{language}",
            "book_title": "{book_title}",
            "chapter_title": "{chapter_title}"
        }
}

In [ ]:
# EPPO_features = Features({
    
#     'id': Value(dtype='string', id=None),
#     'question': Value(dtype='string', id=None),
#     'options': Sequence(feature=Value("string")),
#     'answer': Value(dtype='string', id=None),
    
#     'image': Image(decode=True, id=None),
    
#     'taxon_difficulty': Value(dtype='int32', id=None), # how common is the taxon in the picture?
#     'options_difficulty': Value(dtype='int32', id=None),
#     'kingdom': Value(dtype='string', id=None),
    
#     'category': Value(dtype='string', id=None), # Taxonomy, Growth Stage, Common Name
#     'question_template' : Value(dtype='string', id=None), # scientific_name, common_name
#     'question_type': Value(dtype='string', id=None), # multiple choice, open ended
    
#     'metadata': {'source': Value(dtype='string'),
#                                   'license': Value(dtype='string'),
#                                   'image_url': Value(dtype='string'),
#                                   'language': Value(dtype='string'),
#                                   'verbose_answer': Value(dtype='string'),
#                                   "eppo_code": Value(dtype='string'),
#                                   "gbif_key": Value(dtype='string'),
#                                   "common_name_language": Value(dtype='string') 
#                                   }
#     })

In [ ]:
# GBIF_features = Features({
    
#     'id': Value(dtype='string', id=None),
#     'question': Value(dtype='string', id=None),
#     'options': Sequence(feature=Value("string")),
#     'answer': Value(dtype='string', id=None),
 
#     'image_1': Image(decode=True, id=None),
#     'image_2': Image(decode=True, id=None),
#     'image_3': Image(decode=True, id=None),
#     'image_4': Image(decode=True, id=None),
#     'image_5': Image(decode=True, id=None),
    
#     'options_difficulty': Value(dtype='int32', id=None), # previously topic_difficulty
    
#     'region': Value(dtype='string', id=None),
#     'event_date': Value(dtype='string', id=None),
    
#     # Category can be Horticulture, tropical plants etc
#     #'category': Value(dtype='string', id=None), #previously subfield # Plant Science, Pests, Taxonomy
#     'question_template' :  Value(dtype='string', id=None),
#     'question_type': Value(dtype='string', id=None), # multiple choice, open ended 
    
#     'metadata': {'source': Value(dtype='string'),
#                                   'license': Value(dtype='string'),
#                                   'image_url': Sequence(feature=Value("string")),
#                                   "gbif_id": Value(dtype='string'),
#                                   "eppo_codes": Sequence(feature=Value("string")),
#                                   "gbif_taxon_key": Value(dtype='string'),
#                                   'language': Value(dtype='string'),
#                                   'verbose_answer': Value(dtype='string')
#                                   }
#     })

In [39]:
GBIF_row = {
    "id": "_GBIF_{number}",
    "question": "{question}",
    "options": ["{option_1}", "{option_2}", "{option_3}", "{option_4}"],  # List of options
    "answer": "{answer}",
    "image_1": "{image_path_1}",  # Path to image 1
    "image_2": "{image_path_2}",  # Path to image 2
    "image_3": "{image_path_3}",  # Path to image 3
    "image_4": "{image_path_4}",  # Path to image 4
    "image_5": "{image_path_5}",  # Path to image 5
    "options_difficulty": 0,  # Integer
    "region": "{region}",  # Example: Europe, Asia
    "event_date": "{event_date}",  # Example: 2024-12-16
    "question_template": "{question_template}",  # Example: common_name, scientific_name
    "question_type": "{question_type}",  # Example: multiple choice, open ended
    "metadata":
        {
            "source": "{source}",
            "license": "{license}",
            "image_url": ["{image_url_1}", "{image_url_2}"],  # List of image URLs
            "gbif_id": "{gbif_id}",
            "eppo_codes": ["{eppo_code_1}", "{eppo_code_2}"],  # List of EPPO codes
            "gbif_taxon_key": "{gbif_taxon_key}",
            "language": "{language}",
            "verbose_answer": "{verbose_answer}"
        }
}

In [ ]:
# WikiHow_features = Features({
    
#  'id': Value(dtype='string', id=None),
#  'question': Value(dtype='string', id=None),
#  'options': Sequence(feature=Value("string")),
#  'answer': Value(dtype='string', id=None),
 
#  'options_difficulty': Value(dtype='int32', id=None),
 
#  'category': Value(dtype='string', id=None), # TODO
#  'question_type': Value(dtype='string', id=None), # multiple choice, open ended
#  'question_template' : Value(dtype='string', id=None), # previously metadata[questiontype]
 
#  'metadata': {'source': Value(dtype='string'),
#                                'url': Value(dtype='string'),
#                                'language': Value(dtype='string'),
#                                'verbose_answer': Value(dtype='string')
#                                }
#  })

In [41]:
WikiHow_row = {
    "id": "_WikiHow_{number}",
    "question": "{question}",
    "options": ["{option_1}", "{option_2}", "{option_3}", "{option_4}"],  # List of options
    "answer": "{answer}",
    "options_difficulty": 0,  # Integer
    "category": "{category}",  # Example: TODO
    "question_type": "{question_type}",  # Example: multiple choice, open ended
    "question_template": "{question_template}",  # Example: questiontype
    "metadata":
        {
            "source": "{source}",
            "url": "{url}",
            "language": "{language}",
            "verbose_answer": "{verbose_answer}"
        }
}

### Create dataframes

### Split and Save

In [14]:
import random
RANDOM_SEED = 888 # lottomatica

def create_split(data, dev_size=16, test_size_ratio=0.8, random_seed=RANDOM_SEED):
    # data is rows as a list of dictionaries 
    random.seed(random_seed)
    # Shuffle the data for randomness
    data = deepcopy(data)
    random.shuffle(data)
    
    # Split the data into dev, test, and validation
    dev_split = data[:dev_size]
    remaining_data = data[dev_size:]
    test_size = int(len(remaining_data) * test_size_ratio)
    test_split = remaining_data[:test_size]
    validation_split = remaining_data[test_size:]
    
    split_dict = {'dev': dev_split, 'test': test_split, 'validation': validation_split}
        
    return split_dict

def rename_split(split_dict):
    # split_dict is a dictionary {split_name: rows}
    # renames row['id'] adding split_name and increasing id
    
    renamed = {k: v for k, v in split_dict.items()}
    
    for split, rows in split_dict.items():
        for i, row in enumerate(rows):
            row['id'] = f"{split}_{row['id']}_{i+1}"
            
    return renamed

def split_and_save(all_rows, root_folder='/workdir/AGRIVQA/prova/', max_len=None, random_seed=RANDOM_SEED):
    dataset_name = all_rows[0]['id']
    
    split_dict = create_split(all_rows, random_seed=random_seed)
    split_dict = rename_split(split_dict)
       
    for split in split_dict:
        rows = split_dict[split]

        if not max_len:
            max_rows = len(rows)
        else:
            max_rows = max_len
            
        for i in range(0, len(rows), max_rows):
            dataset = Dataset.from_list(rows[i:i+max_rows])
            num_files = 1 if len(rows) == max_rows else len(rows) // max_rows + 1
            file_name = f'{split}-{str(i//max_rows).zfill(5)}-of-{str(num_files).zfill(5)}.parquet'
            file_path = os.path.join(root_folder, dataset_name, file_name)
            
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            dataset.to_parquet(file_path)   
    

### AgriExam


In [33]:
AgriExam_row = {
    "id": "_AgriExam_{number}",
    "question": "{question}",
    "options": ["{option_1}", "{option_2}", "{option_3}", "{option_4}"],  # Assuming options are strings in a list
    "answer": "{answer}",
    "category": "{category}",  # Matches the 'category' field
    "question_type": "{question_type}",  # Matches the 'question_type' field
    "metadata":
        {
            "source": "{source}",
            "license": "{license}",
            "url": "{url}",
            "language": "{language}",
            "verbose_answer": "{verbose_answer}"
        }
}

In [286]:
AgriExam_rows = []

AgriExam_data = load_json('/workdir/agriexam_category_dictionary.json')

for category, questions in AgriExam_data.items():

    for q in tqdm(questions):
        AgriExam_row['id'] = f"AgriExam"
        AgriExam_row['question'] = q['question']
        options = eval(q['options'])
        if len(options) == 0:
            options = []
        AgriExam_row['options'] = options
            
        # Placeholder for explanation or any additional info
        AgriExam_row['answer'] = q['answer']  # Assuming the correct answer is in q.correct_answer
        
        # Placeholder for category if applicable, adjust based on your requirements
        AgriExam_row['category'] = category  # Replace with actual category logic if needed
        
        # Placeholder for question_type logic: assuming it's a multiple-choice or open-ended type
        if options:
            AgriExam_row['question_type'] = 'multiple-choice'
        else:
            AgriExam_row['question_type'] = 'open-ended'
        
        # Metadata section: Assuming placeholders for now
        metadata = eval(q['metadata'])
        
        AgriExam_row['metadata']['source'] = metadata['source']  # Replace with actual source if needed
        AgriExam_row['metadata']['license'] = metadata['license']
        AgriExam_row['metadata']['url'] = metadata['url']
        AgriExam_row['metadata']['language'] = metadata['language']
        AgriExam_row['metadata']['verbose_answer'] = metadata['verbose_answer']

        # Appending the row to rows
        AgriExam_rows.append(deepcopy(AgriExam_row))

  0%|          | 0/347 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:00<00:00, 19687.87it/s]


In [323]:
split_and_save(AgriExam_rows)
path = '/workdir/AGRIVQA/prova'
dataset_name = 'AgriExam'

dataset_dict = load_dataset('parquet', data_files={
    'dev': f'{path}/{dataset_name}/dev-00000-of-00001.parquet',
    'test': f'{path}/{dataset_name}/test-00000-of-00001.parquet',
    'validation': f'{path}/{dataset_name}/validation-00000-of-00001.parquet'
})

dataset_dict['dev'][0]

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 529.65ba/s]
Generating dev split: 16 examples [00:00, 4733.98 examples/s]
Generating test split: 3625 examples [00:00, 321288.84 examples/s]
Generating validation split: 907 examples [00:00, 206538.56 examples/s]


{'id': 'dev_AgriExam_1',
 'question': 'Which of the following states is the largest producer of annual flowers’ seeds?',
 'options': ['Punjab', 'Karnataka', 'Tamil Nadu', 'Kerala'],
 'answer': 'A',
 'category': 'Horticulture and Ornamental Plants',
 'question_type': 'multiple-choice',
 'metadata': {'language': 'English',
  'license': '',
  'source': 'AgriExam',
  'url': 'https://www.agriexam.com/horticulture-jrf-2020',
  'verbose_answer': 'Punjab'}}

### Agri500P

In [278]:
Agri500P_row = {
    "id": "_Agri500P_{number}",
    "question": "{question}",
    "answer": "{answer}",
    "context": "{context}",
    "category": "{category}",  # Example: Plant Science, Pests, Taxonomy
    "question_type": "{question_type}",  # Example: multiple choice, open ended
    "metadata":
        {
            "source": "{source}",
            "license": "{license}",
            "language": "{language}",
            "book_title": "{book_title}",
            "chapter_title": "{chapter_title}"
        }
}

In [ ]:
task_name = '500P'

dataset_name = "parquet"
data_files = {
    "dev": "/workdir/important_datasets/AGRIVQA/"+task_name+"/dev-00000-of-00001.parquet",
    "test": "/workdir/important_datasets/AGRIVQA/"+task_name+"/test-00000-of-00001.parquet",
    "validation": "/workdir/important_datasets/AGRIVQA/"+task_name+"/validation-00000-of-00001.parquet"
}

dataset = load_dataset(dataset_name,data_files=data_files)

Agri500P_dataset = concatenate_datasets([dataset['dev'], dataset['test'], dataset['validation']])


In [ ]:
Agri500P_rows = []

pt_en_title=load_json('/workdir/translate_title.json')
en_pt_title = {en:pt for pt,en in pt_en_title.items()}

Agri500P_category = load_json('/workdir/500P_categories.json')
theme_category_dict = {theme:category  for category,theme_list in Agri500P_category.items() for theme in theme_list}


for q in tqdm(Agri500P_dataset):
    Agri500P_row['id'] = f"Agri500P"
    Agri500P_row['question'] = q['question']
        
    # Placeholder for explanation or any additional info
    Agri500P_row['answer'] = q['answer']  # Assuming the correct answer is in q.correct_answer
    
    # Metadata section: Assuming placeholders for now
    metadata = eval(q['metadata'])
    
    # Placeholder for category if applicable, adjust based on your requirements
    
    book_title = metadata['book_title'].replace(' – The Producer Asks, Embrapa Answers', 's')
    chapter_title = metadata['chapter_title'].replace('Strawberry production in greenhouses is a more common topic, but I assume you meant... ', '')
    
    category = theme_category_dict[f"{book_title}: {chapter_title}"]
    
    Agri500P_row['category'] = category  # Replace with actual category logic if needed
    
    Agri500P_row['context'] = f"This question is sourced from the book titled '{book_title}', specifically found in the chapter '{chapter_title}'."
    
    # Placeholder for question_type logic: assuming it's a multiple-choice or open-ended type
    if options:
        Agri500P_row['question_type'] = 'multiple-choice'
    else:
        Agri500P_row['question_type'] = 'open-ended'
    
    
    
    Agri500P_row['metadata']['source'] = '500 Perguntas e 500 Respostas: ' + en_pt_title[book_title]  # Replace with actual source if needed
    Agri500P_row['metadata']['license'] = metadata['license']
    Agri500P_row['metadata']['language'] = metadata['language']
    Agri500P_row['metadata']['book_title'] = metadata['book_title']
    Agri500P_row['metadata']['chapter_title'] = metadata['chapter_title']
    # Appending the row to rows
    Agri500P_rows.append(deepcopy(Agri500P_row))

100%|██████████| 20217/20217 [00:03<00:00, 5269.03it/s]


In [326]:
path = '/workdir/AGRIVQA/prova'
split_and_save(Agri500P_rows, root_folder=path)
dataset_name = 'Agri500P'

dataset_dict = load_dataset('parquet', data_files={
    'dev': f'{path}/{dataset_name}/dev-00000-of-00001.parquet',
    'test': f'{path}/{dataset_name}/test-00000-of-00001.parquet',
    'validation': f'{path}/{dataset_name}/validation-00000-of-00001.parquet'
})

dataset_dict['dev'][0]

Creating parquet from Arrow format: 100%|██████████| 5/5 [00:00<00:00, 397.70ba/s]
Generating dev split: 16 examples [00:00, 3582.58 examples/s]
Generating test split: 16160 examples [00:00, 200159.92 examples/s]
Generating validation split: 4041 examples [00:00, 166812.81 examples/s]


{'id': 'dev_Agri500P_1',
 'question': 'Where can the seed washing equipment be acquired to enable oil extraction?',
 'answer': 'The equipment for processing passion fruit pulp waste for seed purification was developed and patented by the State University of the North Fluminense (Uenf), in partnership with Embrapa. Therefore, if someone is interested in acquiring the aforementioned equipment, they simply need to contact the aforementioned institutions at the addresses listed in the base of this page^14,15.',
 'context': "This question is sourced from the book titled 'Passion Fruit', specifically found in the chapter 'Marketing and Value Addition'.",
 'category': 'Nutrition, Health Benefits, and Food Processing',
 'question_type': 'multiple-choice',
 'metadata': {'book_title': 'Passion Fruit',
  'chapter_title': 'Marketing and Value Addition',
  'language': 'English',
  'license': '',
  'source': '500 Perguntas e 500 Respostas: Maracujá'}}

### EPPO



In [6]:
task_name = 'EPPO'

dataset_name = "parquet"
data_files = {
    "dev": "/workdir/important_datasets/AGRIVQA/"+task_name+"/dev-00000-of-00001.parquet",
    "test": "/workdir/important_datasets/AGRIVQA/"+task_name+"/test-00000-of-00001.parquet",
    "validation": "/workdir/important_datasets/AGRIVQA/"+task_name+"/validation-00000-of-00001.parquet"
}

dataset = load_dataset(dataset_name,data_files=data_files)

EPPO_dataset = concatenate_datasets([dataset['dev'], dataset['test'], dataset['validation']])


In [10]:
template_dict = {
    'common_name':'Common Name',
    'damage_cause': 'Taxonomy',
    'genus_name': 'Taxonomy',
    'growth_stage': 'Growth Stage',
    'scientific_name': 'Taxonomy',
    'weed_identification': 'Taxonomy'
}

kingdoms = [
 'Archaea',
 'Bacteria',
 'Chromista',
 'Viruses_and_viroids',
 'Fungi',
 'Protista',
 'Plantae',
 'Animalia']

kingdom_eppo_dict = { k: list(load_json(f'/workdir/important_datasets/EPPO_to_GBIF/{k}_EPPO_to_GBIF.json')) for k in kingdoms}

In [11]:
EPPO_row = {
    "id": "EPPO",
    "question": "{question}",
    "options": ["{option_1}", "{option_2}", "{option_3}", "{option_4}"],  # List of options
    "answer": "{answer}",
    "image": "{image_path}",  # Path to the image
    "taxon_rank": 0,  # Integer
    "options_difficulty": 0,  # Integer
    "kingdom": "{kingdom}",  # Example: Plantae, Animalia
    "category": "{category}",  # Example: Taxonomy, Growth Stage, Common Name
    "question_template": "{question_template}",  # Example: scientific_name, common_name
    "question_type": "{question_type}",  # Example: multiple choice, open ended
    "metadata":
        {
            "source": "{source}",
            "license": "{license}",
            "image_url": "{image_url}",
            "language": "{language}",
            "verbose_answer": "{verbose_answer}",
            "eppo_code": "{eppo_code}",
            "gbif_key": "{gbif_key}",
            "common_name_language": "{common_name_language}"
        }
}

In [12]:
EPPO_rows = []

OPTIONS = ['A','B','C','D','E']

for q in tqdm(EPPO_dataset):
    EPPO_row['id'] = f"EPPO"
    EPPO_row['question'] = q['question']
        
    # Placeholder for explanation or any additional info
    EPPO_row['answer'] = q['answer']  # Assuming the correct answer is in q.correct_answer
    
    options = eval(q['options'])
    if len(options) == 0:
            options = []
    EPPO_row['options'] = options
    
    EPPO_row['image'] = q['image_1']
    EPPO_row['options_difficulty'] = int(q['options_difficulty'])
    # Metadata section: Assuming placeholders for now
    metadata = eval(q['metadata'])
    
    kingdom = metadata['kingdom']
    EPPO_row['kingdom'] = kingdom
    EPPO_row['taxon_rank'] = kingdom_eppo_dict[kingdom].index(metadata['eppo_code'])
    
    # Placeholder for category if applicable, adjust based on your requirements
    EPPO_row["question_template"] = metadata['tag']
    
    EPPO_row['category'] = template_dict[metadata['tag']]  # Replace with actual category logic if needed
    
    
    #EPPO_row['context'] = #f"This question is sourced from the book titled '{book_title}', specifically found in the chapter '{chapter_title}'."
    
    # Placeholder for question_type logic: assuming it's a multiple-choice or open-ended type
    if options:
        EPPO_row['question_type'] = 'multiple-choice'
    else:
        EPPO_row['question_type'] = 'open-ended'
    
    
    EPPO_row['metadata']['source'] = 'EPPO'  # Replace with actual source if needed
    EPPO_row['metadata']['license'] = metadata['license']
    EPPO_row['metadata']['language'] = metadata['language']
    EPPO_row['metadata']['image_url'] = metadata['url']
    EPPO_row['metadata']['verbose_answer'] = options[OPTIONS.index(q['answer'])]
    EPPO_row['metadata']['eppo_code'] = metadata['eppo_code']
    EPPO_row['metadata']['gbif_key'] = metadata['gbif_key']
    EPPO_row['metadata']['common_name_language'] = metadata['common_name_language']
    # Appending the row to rows
    EPPO_rows.append(deepcopy(EPPO_row))

100%|██████████| 20648/20648 [02:27<00:00, 139.70it/s]


In [15]:
d = Dataset.from_list(EPPO_rows)

In [16]:
d

Dataset({
    features: ['id', 'question', 'options', 'answer', 'image', 'taxon_rank', 'options_difficulty', 'kingdom', 'category', 'question_template', 'question_type', 'metadata'],
    num_rows: 20648
})

In [13]:
root_folder = '/workdir/important_datasets/AGRIVQA_v2'
split_and_save(EPPO_rows, root_folder=root_folder, max_len=1000)
dataset_name = 'EPPO'

dataset_dict = load_dataset('parquet', data_files={
    'dev': f'{root_folder}/{dataset_name}/dev-00000-of-00001.parquet',
    'test': f'{root_folder}/{dataset_name}/test-00000-of-00001.parquet',
    'validation': f'{root_folder}/{dataset_name}/validation-00000-of-00001.parquet'
})

dataset_dict['dev'][0]

KeyboardInterrupt: 

### GBIF

In [368]:
task_name = 'Identification'

dataset_name = "parquet"
data_files = {
    "dev": "/workdir/important_datasets/AGRIVQA/"+task_name+"/dev-00000-of-00001.parquet",
    "test": "/workdir/important_datasets/AGRIVQA/"+task_name+"/test-00000-of-00001.parquet",
    "validation": "/workdir/important_datasets/AGRIVQA/"+task_name+"/validation-00000-of-00001.parquet"
}

dataset = load_dataset(dataset_name,data_files=data_files)

EPPO_dataset = concatenate_datasets([dataset['dev'], dataset['test'], dataset['validation']])


Generating dev split: 16 examples [00:00, 191.98 examples/s]
Generating test split: 9 examples [00:00, 158.35 examples/s]
Generating validation split: 75 examples [00:00, 335.71 examples/s]


In [ ]:
template_dict = {
    'common_name':'Common Name',
    'damage_cause': 'Taxonomy',
    'genus_name': 'Taxonomy',
    'growth_stage': 'Growth Stage',
    'scientific_name': 'Taxonomy',
    'weed_identification': 'Taxonomy' 
}

In [ ]:
kingdoms = [
 'Archaea',
 'Bacteria',
 'Chromista',
 'Viruses_and_viroids',
 'Fungi',
 'Protista',
 'Plantae',
 'Animalia']
kingdom_eppo_dict = { k: list(load_json(f'/workdir/important_datasets/EPPO_to_GBIF/{k}_EPPO_to_GBIF.json')) for k in kingdoms}

In [ ]:
EPPO_dataset[0]

{'id': 'dev__EPPO_1',
 'question': 'What is the common name in English of this plant? <image 1>',
 'options': "['Indian ginseng', 'violet tube flower', 'sticky tailflower', 'tomato']",
 'explanation': None,
 'image_1': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1365>,
 'image_2': None,
 'image_3': None,
 'image_4': None,
 'image_5': None,
 'img_type': "['Picture']",
 'answer': 'D',
 'options_difficulty': '4',
 'question_type': 'multiple-choice',
 'subfield': 'Plantae / common_name',
 'metadata': '{"source": "EPPO", "author": "EPPO", "license": "", "url": "https://gd.eppo.int/media/data/taxon/L/LYPES/pics/1024x0/4984.jpg", "language": "English", "verbose_answer": "{verbose_answer}", "eppo_code": "LYPES", "gbif_key": "2930137", "kingdom": "Plantae", "tag": "common_name", "common_name_language": "English"}'}

In [ ]:
EPPO_row = {
    "id": "EPPO",
    "question": "{question}",
    "options": ["{option_1}", "{option_2}", "{option_3}", "{option_4}"],  # List of options
    "answer": "{answer}",
    "image": "{image_path}",  # Path to the image
    "taxon_rank": 0,  # Integer
    "options_difficulty": 0,  # Integer
    "kingdom": "{kingdom}",  # Example: Plantae, Animalia
    "category": "{category}",  # Example: Taxonomy, Growth Stage, Common Name
    "question_template": "{question_template}",  # Example: scientific_name, common_name
    "question_type": "{question_type}",  # Example: multiple choice, open ended
    "metadata":
        {
            "source": "{source}",
            "license": "{license}",
            "image_url": "{image_url}",
            "language": "{language}",
            "verbose_answer": "{verbose_answer}",
            "eppo_code": "{eppo_code}",
            "gbif_key": "{gbif_key}",
            "common_name_language": "{common_name_language}"
        }
}

In [ ]:
EPPO_rows = []

OPTIONS = ['A','B','C','D','E']

for q in tqdm(EPPO_dataset):
    EPPO_row['id'] = f"EPPO"
    EPPO_row['question'] = q['question']
        
    # Placeholder for explanation or any additional info
    EPPO_row['answer'] = q['answer']  # Assuming the correct answer is in q.correct_answer
    
    options = eval(q['options'])
    if len(options) == 0:
            options = []
    EPPO_row['options'] = options
    
    EPPO_row['image'] = q['image_1']
    EPPO_row['options_difficulty'] = int(q['options_difficulty'])
    # Metadata section: Assuming placeholders for now
    metadata = eval(q['metadata'])
    
    kingdom = metadata['kingdom']
    EPPO_row['kingdom'] = kingdom
    EPPO_row['taxon_rank'] = kingdom_eppo_dict[kingdom].index(metadata['eppo_code'])
    
    # Placeholder for category if applicable, adjust based on your requirements
    EPPO_row["question_template"] = metadata['tag']
    
    EPPO_row['category'] = template_dict[metadata['tag']]  # Replace with actual category logic if needed
    
    
    #EPPO_row['context'] = #f"This question is sourced from the book titled '{book_title}', specifically found in the chapter '{chapter_title}'."
    
    # Placeholder for question_type logic: assuming it's a multiple-choice or open-ended type
    if options:
        EPPO_row['question_type'] = 'multiple-choice'
    else:
        EPPO_row['question_type'] = 'open-ended'
    
    
    EPPO_row['metadata']['source'] = 'EPPO'  # Replace with actual source if needed
    EPPO_row['metadata']['license'] = metadata['license']
    EPPO_row['metadata']['language'] = metadata['language']
    EPPO_row['metadata']['image_url'] = metadata['url']
    EPPO_row['metadata']['verbose_answer'] = options[OPTIONS.index(q['answer'])]
    EPPO_row['metadata']['eppo_code'] = metadata['eppo_code']
    EPPO_row['metadata']['gbif_key'] = metadata['gbif_key']
    EPPO_row['metadata']['common_name_language'] = metadata['common_name_language']
    # Appending the row to rows
    EPPO_rows.append(deepcopy(EPPO_row))

100%|██████████| 20648/20648 [02:43<00:00, 126.65it/s]


In [ ]:
split_and_save(EPPO_rows)
path = '/workdir/important_datasets/AGRIVQA_v2'
dataset_name = 'EPPO'

dataset_dict = load_dataset('parquet', data_files={
    'dev': f'{path}/{dataset_name}/dev-00000-of-00001.parquet',
    'test': f'{path}/{dataset_name}/test-00000-of-00001.parquet',
    'validation': f'{path}/{dataset_name}/validation-00000-of-00001.parquet'
})

dataset_dict['dev'][0]

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 529.65ba/s]
Generating dev split: 16 examples [00:00, 4733.98 examples/s]
Generating test split: 3625 examples [00:00, 321288.84 examples/s]
Generating validation split: 907 examples [00:00, 206538.56 examples/s]


{'id': 'dev_AgriExam_1',
 'question': 'Which of the following states is the largest producer of annual flowers’ seeds?',
 'options': ['Punjab', 'Karnataka', 'Tamil Nadu', 'Kerala'],
 'answer': 'A',
 'category': 'Horticulture and Ornamental Plants',
 'question_type': 'multiple-choice',
 'metadata': {'language': 'English',
  'license': '',
  'source': 'AgriExam',
  'url': 'https://www.agriexam.com/horticulture-jrf-2020',
  'verbose_answer': 'Punjab'}}